# Podjetje, varianta 2

In [1]:
import os
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func, create_engine
from sqlalchemy.orm import relationship, backref, declarative_base, sessionmaker

In [2]:
Base = declarative_base()


class Oddelek(Base):
    __tablename__ = 'oddelek'

    id = Column(Integer, primary_key=True)
    naziv = Column(String)

    # Ime razreda lahko podamo kot niz (ker še ni definirano)
    seznam_zaposlenih = relationship('Zaposleni', secondary='povezava',
                                     back_populates='seznam_oddelkov')

    def __repr__(self):
        return f"Oddelek[{self.id}, {self.naziv}]"

    
class Zaposleni(Base):
    __tablename__ = 'zaposleni'

    id = Column(Integer, primary_key=True)
    ime = Column(String)
    datum_zaposlitve = Column(DateTime, default=func.now())
    seznam_oddelkov = relationship(Oddelek, secondary='povezava',
                                   back_populates='seznam_zaposlenih')

    def __repr__(self):
        return f"Zaposleni[{self.id}, {self.ime}, {self.datum_zaposlitve}]"


class Povezava(Base):
    __tablename__ = 'povezava'

    oddelek_id = Column(Integer, ForeignKey('oddelek.id'), primary_key=True)
    zaposleni_id = Column(Integer, ForeignKey('zaposleni.id'), primary_key=True)

Povežimo se na bazo - če že obstaja, jo bomo najprej pobrisali.

In [3]:
baza = 'podjetje2.db'
if os.path.exists(baza):
    os.remove(baza)

engine = create_engine(f'sqlite:///{baza}', echo=True)

In [4]:
Base.metadata.create_all(engine)
DBSessionMaker = sessionmaker(bind=engine)
session = DBSessionMaker()

2025-06-05 10:53:05,212 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:53:05,212 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("oddelek")
2025-06-05 10:53:05,213 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:53:05,214 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("oddelek")
2025-06-05 10:53:05,214 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:53:05,215 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("zaposleni")
2025-06-05 10:53:05,215 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:53:05,216 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("zaposleni")
2025-06-05 10:53:05,216 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:53:05,217 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("povezava")
2025-06-05 10:53:05,217 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:53:05,218 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("povezava")
2025-06-05 10:53:05,218 INFO sqlalchemy.engine.Engine [raw s

Ustvarimo dva oddelka in tri zaposlene.

In [5]:
IT = Oddelek(naziv="IT")
finance = Oddelek(naziv="Finance")
metka = Zaposleni(ime="Metka")
janez = Zaposleni(ime="Janez")
katka = Zaposleni(ime="Katka")

Dodajmo zaposlene v oddelke in zapišimo v bazo.

In [6]:
katka.seznam_oddelkov.append(finance)
finance.seznam_zaposlenih.append(metka)
janez.seznam_oddelkov.append(IT)

In [7]:
session.add(IT)
session.add(finance)
session.add(janez)
session.add(metka)
session.add(katka)
session.commit()

2025-06-05 10:54:01,035 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:54:01,036 INFO sqlalchemy.engine.Engine INSERT INTO oddelek (naziv) VALUES (?)
2025-06-05 10:54:01,037 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ('IT',)
2025-06-05 10:54:01,038 INFO sqlalchemy.engine.Engine INSERT INTO oddelek (naziv) VALUES (?)
2025-06-05 10:54:01,039 INFO sqlalchemy.engine.Engine [cached since 0.002363s ago] ('Finance',)
2025-06-05 10:54:01,040 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve) VALUES (?, CURRENT_TIMESTAMP)
2025-06-05 10:54:01,040 INFO sqlalchemy.engine.Engine [generated in 0.00036s] ('Janez',)
2025-06-05 10:54:01,041 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve) VALUES (?, CURRENT_TIMESTAMP)
2025-06-05 10:54:01,041 INFO sqlalchemy.engine.Engine [cached since 0.001339s ago] ('Katka',)
2025-06-05 10:54:01,042 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve) VALUES (?, CURRENT

Izpišimo Janezove oddelke.

In [8]:
janez.seznam_oddelkov

2025-06-05 10:55:12,180 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:55:12,184 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve 
FROM zaposleni 
WHERE zaposleni.id = ?
2025-06-05 10:55:12,185 INFO sqlalchemy.engine.Engine [generated in 0.00122s] (1,)
2025-06-05 10:55:12,187 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek, povezava 
WHERE ? = povezava.zaposleni_id AND oddelek.id = povezava.oddelek_id
2025-06-05 10:55:12,187 INFO sqlalchemy.engine.Engine [generated in 0.00046s] (1,)


[Oddelek[1, IT]]

Izpišimo zaposlene na oddelku Finance.

In [9]:
finance.seznam_zaposlenih

2025-06-05 10:55:20,200 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek 
WHERE oddelek.id = ?
2025-06-05 10:55:20,201 INFO sqlalchemy.engine.Engine [generated in 0.00115s] (2,)
2025-06-05 10:55:20,203 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve 
FROM zaposleni, povezava 
WHERE ? = povezava.oddelek_id AND zaposleni.id = povezava.zaposleni_id
2025-06-05 10:55:20,204 INFO sqlalchemy.engine.Engine [generated in 0.00055s] (2,)


[Zaposleni[2, Katka, 2025-06-05 08:54:01],
 Zaposleni[3, Metka, 2025-06-05 08:54:01]]

In [11]:
metka.seznam_oddelkov

[Oddelek[2, Finance]]

Izpišimo zaposlene na oddelku Finance s poizvedbo na oddelkih.

In [12]:
session.query(Oddelek).filter(Oddelek.naziv == 'Finance').one().seznam_zaposlenih

2025-06-05 10:56:23,052 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek 
WHERE oddelek.naziv = ?
2025-06-05 10:56:23,053 INFO sqlalchemy.engine.Engine [generated in 0.00152s] ('Finance',)


[Zaposleni[2, Katka, 2025-06-05 08:54:01],
 Zaposleni[3, Metka, 2025-06-05 08:54:01]]

In [13]:
session.query(Oddelek).filter(Oddelek.naziv == 'IT').one().seznam_zaposlenih

2025-06-05 10:57:15,389 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek 
WHERE oddelek.naziv = ?
2025-06-05 10:57:15,392 INFO sqlalchemy.engine.Engine [cached since 52.34s ago] ('IT',)
2025-06-05 10:57:15,397 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve 
FROM zaposleni, povezava 
WHERE ? = povezava.oddelek_id AND zaposleni.id = povezava.zaposleni_id
2025-06-05 10:57:15,399 INFO sqlalchemy.engine.Engine [cached since 115.2s ago] (1,)


[Zaposleni[1, Janez, 2025-06-05 08:54:01]]

Izpišimo zaposlene na oddelku Finance s poizvedbo na zaposlenih.

In [14]:
session.query(Zaposleni).filter(Zaposleni.seznam_oddelkov.any(Oddelek.naziv == 'Finance')).all()

2025-06-05 10:58:21,995 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve 
FROM zaposleni 
WHERE EXISTS (SELECT 1 
FROM povezava, oddelek 
WHERE zaposleni.id = povezava.zaposleni_id AND oddelek.id = povezava.oddelek_id AND oddelek.naziv = ?)
2025-06-05 10:58:21,996 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ('Finance',)


[Zaposleni[2, Katka, 2025-06-05 08:54:01],
 Zaposleni[3, Metka, 2025-06-05 08:54:01]]